## Request Live data

In [ ]:
import threading
import time
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.ticktype import TickType, TickTypeEnum

class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
    def nextValidId(self, orderId: int):
        
        mycontract = Contract()
        mycontract.symbol = "AAPL"
        mycontract.secType = "STK"
        mycontract.exchange = "SMART"
        mycontract.currency = "USD"
        self.reqMarketDataType(4)
        self.reqMktData(orderId, mycontract, "", 0, 0, [])
        
    def tickPrice(self, reqId, tickType, price, attrib):
        print(f"tickPrice. reqId: {reqId}, tickType: {TickTypeEnum.to_str(tickType)}, price: {price}, attribs: {attrib}")
    # def tickSize(self, reqId, tickType, size):
    #     print(f"tickSize. reqId: {reqId}, tickType: {TickTypeEnum.to_str(tickType)}, size: {size}")
        
app = TestApp()
app.connect("127.0.0.1", 7496, 1000)

def run_loop():
	app.run()

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(3)
app.disconnect()

## Request Historical Market Data

In [ ]:
from ibapi.client import *
from ibapi.wrapper import *
import datetime
class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
    def nextValidId(self, orderId: int):
        
        mycontract = Contract()
        mycontract.symbol = "AAPL"
        mycontract.secType = "STK"
        mycontract.exchange = "SMART"
        mycontract.currency = "USD"
        queryTime = (datetime.datetime.today() - datetime.timedelta(days=180)).strftime("%Y%m%d-%H:%M:%S")
        self.reqHistoricalData(orderId, mycontract, queryTime, "1 D", "1 min", "TRADES", 0, 1, 0, [])
    def historicalData(self, reqId, bar):
        print(f"Historical Data: {bar}")
    def historicalDataEnd(self, reqId, start, end):
        print(f"End of HistoricalData")
        print(f"Start: {start}, End: {end}")
app = TestApp()
app.connect("127.0.0.1", 7496, 1000)
app.run()

## Download Historical Data

In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

import threading
import time
from datetime import datetime
import numpy as np
import pandas as pd

def download_data_daily(stock, start_date="01-01-2017", end_date = datetime.now(), size ='1 day'):

	class IBapi(EWrapper, EClient):
		def __init__(self):
			EClient.__init__(self, self)
			self.data = [] #Initialize variable to store candle

		def historicalData(self, reqId, bar):
			# print(f'Time: {bar.date} Close: {bar.close}')
			self.data.append([bar.date, bar.close])
			
	def run_loop():
		app.run()

	app = IBapi()
	app.connect('127.0.0.1', 7496, 123)

	#Start the socket in a thread
	api_thread = threading.Thread(target=run_loop, daemon=True)
	api_thread.start()

	time.sleep(1) #Sleep interval to allow time for connection to server

	#Create contract object
	eurusd_contract = Contract()
	eurusd_contract.symbol = stock
	eurusd_contract.secType = 'STK'
	eurusd_contract.exchange = 'SMART'
	eurusd_contract.currency = 'USD'

	start_date = datetime.strptime(start_date, '%m-%d-%Y')
	end_date = datetime.now()

	days = (end_date - start_date).days
	print("days",days)
	if days < 365:
		duration = "%d D" % days
	else:
		duration = "%d Y" % np.ceil(days / 365)
	# when getting daily data, look at regular trading hours only
	# to get accurate daily closing prices
	useRTH = True
	# round up current time to midnight for even days
	end_date = end_date.replace(
		hour=0, minute=0, second=0, microsecond=0
	)


	end_date = end_date.strftime(f"%Y%m%d %H:%M:%S UTC")
	# duration = '2 D'
	#Request historical candles

	print("start_date",start_date)
	print("end_date", end_date)
	print("duration", duration)
	app.reqHistoricalData(1, eurusd_contract, end_date, duration, size, 'BID', useRTH, 1, False, [])

	time.sleep(5) #sleep to allow enough time for data to be returned

	# #Working with Pandas DataFrames
	import pandas

	df = pandas.DataFrame(app.data, columns=['DateTime', 'Close'])
	# print(df)
	df['DateTime'] = pandas.to_datetime(df['DateTime'],
               format='%Y%m%d') 
	df.to_csv('data/daily/' + stock + '.csv')  

	app.disconnect()
 


In [2]:
stocks = [
    "AAPL",
    "GOOG",
    "AMZN",
    "NVDA",
    "TSLA",
    "AMD",
    "INTC",
    "ATVI",
    "PYPL",
    "TTD",
    "EA",
    ]
for stock in stocks:
    download_data_daily(
        stock
        )


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


days 2412
start_date 2017-01-01 00:00:00
end_date 20230810 00:00:00 UTC
duration 7 Y


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


days 2412
start_date 2017-01-01 00:00:00
end_date 20230810 00:00:00 UTC
duration 7 Y


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


...

days 2412
start_date 2017-01-01 00:00:00
end_date 20230810 00:00:00 UTC
duration 7 Y


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


days 2412
start_date 2017-01-01 00:00:00
end_date 20230810 00:00:00 UTC
duration 7 Y


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


days 2412
start_date 2017-01-01 00:00:00
end_date 20230810 00:00:00 UTC
duration 7 Y


In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

import threading
import time
from datetime import datetime
import numpy as np
import pandas as pd
def download_data_minutely(stock, duration = '1 W'):

	class IBapi(EWrapper, EClient):
		def __init__(self):
			EClient.__init__(self, self)
			self.data = [] #Initialize variable to store candle

		def historicalData(self, reqId, bar):
			# print(f'Time: {bar.date} Close: {bar.close}')
			self.data.append([bar.date, bar.close])
			
	def run_loop():
		app.run()

	app = IBapi()
	app.connect('127.0.0.1', 7496, 123)

	#Start the socket in a thread
	api_thread = threading.Thread(target=run_loop, daemon=True)
	api_thread.start()

	time.sleep(1) #Sleep interval to allow time for connection to server

	#Create contract object
	eurusd_contract = Contract()
	eurusd_contract.symbol = stock
	eurusd_contract.secType = 'STK'
	eurusd_contract.exchange = 'SMART'
	eurusd_contract.currency = 'USD'

	useRTH = True


	print("duration", duration)
	app.reqHistoricalData(
    	1, 
    	eurusd_contract,
		"", 
		duration, 
		'1 min', 
		'BID', 
		useRTH, 
		1, 
		False, 
		[])

	time.sleep(5) #sleep to allow enough time for data to be returned

	# #Working with Pandas DataFrames
	import pandas

	df = pandas.DataFrame(app.data, columns=['DateTime', 'Close'])
	# print(df)
	df['DateTime'] = pandas.to_datetime(df['DateTime'],
               format='%Y%m%d  %H:%M:%S') 
	df.to_csv('data/minutely/' + stock + '.csv')  

	app.disconnect()
 

In [2]:
stocks = [
    "AAPL",
    "GOOG",
    "AMZN",
    "NVDA",
    "TSLA",
    "AMD",
    "INTC",
    "ATVI",
    "PYPL",
    "TTD",
    "EA",
    ]
for stock in stocks:
    download_data_minutely(
        stock, 
        duration="1 M"
        )


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


duration 1 M


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


duration 1 M


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


...

duration 1 M


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


duration 1 M


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


duration 1 M


In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

import threading
import time
from datetime import datetime
import numpy as np
import pandas as pd
def download_data_15minutely(stock, duration = '1 W'):

	class IBapi(EWrapper, EClient):
		def __init__(self):
			EClient.__init__(self, self)
			self.data = [] #Initialize variable to store candle

		def historicalData(self, reqId, bar):
			# print(f'Time: {bar.date} Close: {bar.close}')
			self.data.append([bar.date, bar.close])
			
	def run_loop():
		app.run()

	app = IBapi()
	app.connect('127.0.0.1', 7496, 123)

	#Start the socket in a thread
	api_thread = threading.Thread(target=run_loop, daemon=True)
	api_thread.start()

	time.sleep(1) #Sleep interval to allow time for connection to server

	#Create contract object
	eurusd_contract = Contract()
	eurusd_contract.symbol = stock
	eurusd_contract.secType = 'STK'
	eurusd_contract.exchange = 'SMART'
	eurusd_contract.currency = 'USD'

	useRTH = True


	print("duration", duration)
	app.reqHistoricalData(
    	1, 
    	eurusd_contract,
		"", 
		duration, 
		'15 mins', 
		'BID', 
		useRTH, 
		1, 
		False, 
		[])

	time.sleep(5) #sleep to allow enough time for data to be returned

	# #Working with Pandas DataFrames
	import pandas

	df = pandas.DataFrame(app.data, columns=['DateTime', 'Close'])
	# print(df)
	df['DateTime'] = pandas.to_datetime(df['DateTime'],
               format='%Y%m%d  %H:%M:%S') 
	df.to_csv('data/15minutely/' + stock + '.csv')  

	app.disconnect()
 

In [2]:
stocks = [
    "AAPL",
    "GOOG",
    "AMZN",
    "NVDA",
    "TSLA",
    "AMD",
    "INTC",
    "ATVI",
    "PYPL",
    "TTD",
    "EA",
    ]
for stock in stocks:
    download_data_15minutely(
        stock, 
        duration="1 Y"
        )


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


duration 1 Y


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


duration 1 Y


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


...

duration 1 Y


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


duration 1 Y


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


duration 1 Y


In [4]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

import threading
import time
from datetime import datetime
import numpy as np
import pandas as pd
def download_data_hourly(stock, duration = '1 W'):

	class IBapi(EWrapper, EClient):
		def __init__(self):
			EClient.__init__(self, self)
			self.data = [] #Initialize variable to store candle

		def historicalData(self, reqId, bar):
			# print(f'Time: {bar.date} Close: {bar.close}')
			self.data.append([bar.date, bar.close])
			
	def run_loop():
		app.run()

	app = IBapi()
	app.connect('127.0.0.1', 7496, 123)

	#Start the socket in a thread
	api_thread = threading.Thread(target=run_loop, daemon=True)
	api_thread.start()

	time.sleep(1) #Sleep interval to allow time for connection to server

	#Create contract object
	eurusd_contract = Contract()
	eurusd_contract.symbol = stock
	eurusd_contract.secType = 'STK'
	eurusd_contract.exchange = 'SMART'
	eurusd_contract.currency = 'USD'

	useRTH = True


	print("duration", duration)
	app.reqHistoricalData(
    	1, 
    	eurusd_contract,
		"", 
		duration, 
		'1 hour', 
		'BID', 
		useRTH, 
		1, 
		False, 
		[])

	time.sleep(5) #sleep to allow enough time for data to be returned

	# #Working with Pandas DataFrames
	import pandas

	df = pandas.DataFrame(app.data, columns=['DateTime', 'Close'])
	# print(df)
	df['DateTime'] = pandas.to_datetime(df['DateTime'],
               format='%Y%m%d  %H:%M:%S') 
	df.to_csv('data/hourly/' + stock + '.csv')  

	app.disconnect()

In [5]:
stocks = [
    "AAPL",
    "GOOG",
    "AMZN",
    "NVDA",
    "META",
    "TSLA",
    "BABA",
    "CRM",
    "AMD",
    "INTC",
    "ATVI",
    "PYPL",
    "TTD",
    "EA",
    ]
for stock in stocks:
    download_data_hourly(
        stock, 
        duration="2 W"
        )


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


duration 2 W


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


duration 2 W


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


...

duration 2 W


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


duration 2 W


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


duration 2 W
